# <center>Data Visualization 2 -  Assignment</center>

-----------------------------------------------------------------------------------------------------------

**1. Dataset description:**

`The World Happiness Report` is a landmark survey of the state of global happiness. This report reviews the state of happiness in the world in 2021. The dataset measures happiness through:
- `economic production`,
- `social support`,
- `life expectancy`,
- `freedom`,
- `absence of corruption` and
- `generosity`.

`Dystopia`: a hypothetical country that has values equal to the world’s lowest national averages for each of the six factors. These dimensions contribute to making life evaluations higher in each country than they are in the Dystopia column.

`Ladder score`: The main scale that I am considering in the analysis. The Cantril ladder asks respondents to think of a ladder, with the best possible life for them being a 10 and the worst possible life being a 0. They are then asked to rate their own current lives on that 0 to 10 scale.

`Social support score`: Another significant measurement I take into account in the analysis. Social support is the national average of the binary responses (0=no, 1=yes) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

- Data source: [Kaggle](https://www.kaggle.com/datasets/ajaypalsinghlo/world-happiness-report-2021/)
- Further information on the dataset: [World Happiness Report](https://worldhappiness.report/ed/2023/world-happiness-trust-and-social-connections-in-times-of-crisis/)

----------------------------------------------------------------------------------------------------------

**2. Workflow:**
- Importing the required libraries and loading data from my [GitHub Repo](https://github.com/zsofiarebeka/DataViz2)
- Data wrangling: renaming columns, filtering and checking for missing values
- Choosing the dimensions and variables for visualization
- Creating a variable and a unique list for Dash filtering purposes
- Creating the dash app and customizing the charts

-----------------------------------------------------------------------------------------------------------

**3. Output:**
- The scatter plot shows the level of happiness compared to GDP per capita by regions
- The bar chart shows the level of support for each country
- The aim of this dashboard is to compare how GDP per capita and the level of social support define the level of happiness of a country
- The aim of the multi dropdown list is to compare more regions
- To see the correct spacing of the charts, please see: http://127.0.0.1:8050/

In [32]:
# Importing required libraries
import pandas as pd
from dash import Dash 
from dash import html, dcc 
from dash import callback, Output, Input
import plotly.express as px

# Importing the dataset from my public GitHub repository
df = pd.read_csv('https://raw.githubusercontent.com/zsofiarebeka/DataViz2/main/world-happiness-report-2021.csv')

# Checking if the dataset loaded correctly
df.head()

# Replacing ' ' with '_' in the column names for a more concise language and an easier filtering
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Filtering the dataframe for Western Europe
filtered_df = df.query('Regional_indicator == "Western Europe"')

# Checking for missing values
df.Country_name.isna().value_counts()

# Creating a regions unique list as we want to have a regions filter
regions = df['Regional_indicator'].unique()
# Sorting the regions to see if everything can be used for filtering
regions.sort()

# Creating a variable to hold the names of all of the available colorscales
colorscales = px.colors.named_colorscales()
# Sorting by alphabetical order
colorscales.sort()


#####################################################################

# Creating the empty Dash app
app = Dash(__name__)

# Creating the dash layout
app.layout = html.Div([
    
    # Creating the header
    html.H1('World Happiness Data Report from 2021', style={'textAlign': 'center', 'fontSize': 30, 'font-family': 'Railway'}),
    
    # Filtering
    # Adding the label for the region search bar
    html.Label('Select the region: ', style={'font-weight': 'bold', 'font-family': 'Railway'}),
    # Creating and customizing a dropdown list, also adding the multi property option
    dcc.Dropdown(id= 'input1', options=regions, value='Western Europe', multi = True, style={'width':'55%', 'font-family': 'Railway'}),
    # Creating the customized color choice dropdown list for the bar chart for aesthetics
    html.Label('Set the color scale of the bar chart:', style={'font-weight': 'bold', "text-align": "center", 'font-family': 'Railway'}),
    dcc.Dropdown(id= 'input2', options=colorscales, value='oranges', style={'width':'35%', 'font-family': 'Railway'}),
    # Visual placeholder for the charts
    dcc.Graph(id = "scatter1", figure = scatter, style = {'display': 'inline-block', 'width': '50%'}),
    dcc.Graph(id ='fig', figure = fig, style = {'display': 'inline-block', 'width': '10%', 'font-family': 'Railway'}),
])

# Defining the callback
@app.callback(
    Output('scatter1', 'figure'),
    Output('fig', 'figure'),
    Input('input1', 'value'),
    Input('input2', 'value'))

# Defining a function to update charts when the filters are altered
def update_chart(_input1, _input2):
    # Filtering the dataset for scatter plot based on selected region
    df_filtered = df.query('Regional_indicator == @_input1')
    
    # Creating the dynamic title and replacing '[]' with '' that comes with the list attribute
    _title = "Showing measure of happiness in: " + str(_input1).replace('[','').replace(']','').replace("'","")
    
   # Creating a scatter plot comparing ln(GDP per capita) and the happiness score of a country
    scatter = px.scatter(df_filtered, x = 'Logged_GDP_per_capita', y = 'Ladder_score',
                         title = _title,
                         text = 'Country_name',
                         labels = ({'Logged_GDP_per_capita' : 'GDP per capita (ln scale)',
                         'Ladder_score' : 'Ladder Score'}),
                         # Creating a trendline to see the tendency of the regions' happiness scores
                         trendline = 'ols',
                         trendline_color_override = 'rgba(255, 165, 0, 1.0)',
                         template = 'simple_white',
                         # adjusting the size of the chart
                         height = 600, width = 800)
    
    # Updating and customizing the axis labels and its attributes (font, color, size etc.)
    scatter.update_traces(textposition= 'top center', marker=dict(color='orange', size = 3))
    scatter.update_layout(title_x = 0.5, font_size = 12, title_font_family = 'Railway')
    
    # Creating a second dynamic title for the bar chart and replacing '[]' with '' due to the list attribute
    _title2 = "Showing the level social support in: " + str(_input1).replace('[','').replace(']','').replace("'","")

    # Creating the bar chart showing the level of social support by country
    fig = px.bar(df_filtered, x = 'Social_support', y = 'Country_name',
                          title = _title2,
                          labels = {'Country_name' : 'Country',
                          'Social_support' : 'Social Support'},
                          template = 'simple_white',
                          # Setting the colors for social support so the trends are seen
                          color = 'Social_support',
                          color_continuous_scale= str(_input2),
                          # Enabling text_auto for bar labelling
                          text_auto = True,
                          height = 650, width = 700)
    
    # Removing y axis title and repositioning the title of the chart
    fig.update_layout(yaxis_title = None, title_x = 0.5, font_size = 12, title_font_family = 'Railway')

    # Update x axes and positioning values outside the bar for visibility
    fig.update_xaxes(visible = False)
    fig.update_traces(textposition = 'outside')
    
    return scatter, fig # Returning the charts to finish the callback function

# Calling the app
if __name__ == '__main__':
    app.run(debug=True)